In [1]:
import consMDP
import json
from decimal import Decimal

In [2]:
file_path = 'NYCstreetnetwork.json'
with open(file_path,'r') as f:
        graph_file = json.load(f)

In [3]:
def convert(g):
    states = []
    actions = dict()

    mdp = consMDP.ConsMDP()

    for node in g["nodes"]:
        if node["action"]:
            actions[node["label"]] = dict()
        else:
            states.append(node)

    for s in states:
        mdp.new_state(s["reload"], s["label"])

    for edge in g["edges"]:
        fr = edge["tail"]
        to = edge["head"]
        if to in actions:
            actions[to]["from"] = fr
            actions[to]["cons"] = edge["consumption"]
        else:
            dist = actions[fr].get("dist")
            to = mdp.state_with_label(to)
            if dist is None:
                actions[fr]["dist"] = dict()
            actions[fr]["dist"][to] = Decimal(f'{edge["probability"]}')

    for label, a in actions.items():
        fr = mdp.state_with_label(a["from"])
        mdp.add_action(fr, a["dist"], label, a["cons"])

    return mdp

In [6]:
%timeit m = convert(graph_file)

84.1 ms ± 1.62 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [4]:
m = convert(graph_file)
type(m)

consMDP.ConsMDP

In [8]:
%%timeit
m.get_minInitCons(recompute=True);

647 ms ± 3.79 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
%%timeit
m.get_minInitCons(30, recompute=True);

361 ms ± 12.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
%%timeit
m.get_safeReloads(100, recompute=True)

737 ms ± 10.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
